In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import psutil

# Function to get CPU utilization and memory consumption
def get_system_status():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

class SVM:
    def __init__(self, C=1.0, tol=0.01, max_iter=100):
        self.C = C
        self.tol = tol
        self.max_iter = max_iter
        self.b = 0
        self.alpha = None
        self.X = None
        self.y = None

    def fit(self, X, y):
        self.X = X
        self.y = y
        n_samples, n_features = X.shape

        # Initialize alpha and b
        self.alpha = np.zeros(n_samples)
        self.b = 0

        # SMO algorithm
        for _ in range(self.max_iter):
            num_changed_alphas = 0
            for i in range(n_samples):
                E_i = self._decision_function(X[i]) - y[i]

                if (y[i] * E_i < -self.tol and self.alpha[i] < self.C) or (y[i] * E_i > self.tol and self.alpha[i] > 0):
                    j = self._select_random_index(i, n_samples)
                    E_j = self._decision_function(X[j]) - y[j]

                    alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]

                    if y[i] != y[j]:
                        L = max(0, self.alpha[j] - self.alpha[i])
                        H = min(self.C, self.C + self.alpha[j] - self.alpha[i])
                    else:
                        L = max(0, self.alpha[i] + self.alpha[j] - self.C)
                        H = min(self.C, self.alpha[i] + self.alpha[j])

                    if L == H:
                        continue

                    eta = 2 * X[i] @ X[j] - X[i] @ X[i] - X[j] @ X[j]
                    if eta >= 0:
                        continue

                    self.alpha[j] -= y[j] * (E_i - E_j) / eta
                    self.alpha[j] = np.clip(self.alpha[j], L, H)

                    if abs(self.alpha[j] - alpha_j_old) < 1e-5:
                        continue

                    self.alpha[i] += y[i] * y[j] * (alpha_j_old - self.alpha[j])

                    b1 = self.b - E_i - y[i] * (self.alpha[i] - alpha_i_old) * X[i] @ X[i].T - y[j] * (self.alpha[j] - alpha_j_old) * X[i] @ X[j].T
                    b2 = self.b - E_j - y[i] * (self.alpha[i] - alpha_i_old) * X[i] @ X[j].T - y[j] * (self.alpha[j] - alpha_j_old) * X[j] @ X[j].T

                    if 0 < self.alpha[i] < self.C:
                        self.b = b1
                    elif 0 < self.alpha[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2

                    num_changed_alphas += 1

            if num_changed_alphas == 0:
                break

    def _decision_function(self, x):
        return (self.alpha * self.y) @ (self.X @ x.T) + self.b

    def _select_random_index(self, i, n_samples):
        j = i
        while j == i:
            j = np.random.randint(0, n_samples)
        return j

    def predict(self, X):
        return np.sign(self._decision_function(X))

# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=True)
X, y = mnist.data.to_numpy().astype('float32'), mnist.target.astype('int')

# Normalize the pixel values to be in the range [0, 1]
X /= 255.0

# Convert labels to binary (1 for digit 1, -1 for other digits)
y = np.where(y == 1, 1, -1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model = SVM(C=1.0, tol=0.01, max_iter=100)

# Monitoring overall system status
overall_cpu_utilization = []
overall_memory_utilization = []

# Training the SVM model
start_time = time.time()

svm_model.fit(X_train, y_train)

# Monitor system status and log at the start
cpu_percent, memory_percent = get_system_status()
overall_cpu_utilization.append(cpu_percent)
overall_memory_utilization.append(memory_percent)
log_list = [{'Algorithm': 'SVM', 'CPU Utilization': cpu_percent, 'Memory Usage (%)': memory_percent, 'Training Time': 0.0}]

# Calculate the time taken for training at the end
end_time = time.time()
training_time = end_time - start_time
log_list[0]['Training Time'] = training_time

# Print the overall CPU and memory utilization
print(f"Overall CPU Utilization: {np.mean(overall_cpu_utilization)}%")
print(f"Overall Memory Usage: {np.mean(overall_memory_utilization)}%")

# Print the time taken for training
print(f"Time taken for training: {training_time:.2f} seconds")

# Make predictions on the test set
svm_predictions = svm_model.predict(X_test)

# Evaluate accuracy
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy}")


KeyboardInterrupt: 